# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

In [39]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [62]:
%%pycodestyle
import math


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2,
                 min_samples_leaf=1, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        l_w = l_s / (l_s + r_s)

        left_gini = (1 - ((l_c / l_s)**2).sum(axis=1, keepdims=True))
        right_gini = (1 - ((r_c / r_s)**2).sum(axis=1, keepdims=True))
        return 1 - l_w * left_gini - (1 - l_w) * right_gini

    def __entropy(self, l_c, l_s, r_c, r_s):
        p_l = l_c / l_s
        p_r = r_c / r_s
        h_l = -(p_l * np.nan_to_num(np.log2(p_l))).sum(axis=1, keepdims=True)
        h_r = -(p_r * np.nan_to_num(np.log2(p_r))).sum(axis=1, keepdims=True)
        l_w = l_s / (l_s + r_s)
        return l_w * h_l + (1 - l_w) * h_r

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_w = l_s / (l_s + r_s)

        left_m = (1 - np.max((l_c / l_s), axis=1, keepdims=True))
        right_m = (1 - np.max((r_c / r_s), axis=1, keepdims=True))
        return l_w * left_m + (1 - l_w) * right_m

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:math.ceil(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:math.ceil(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # сортировка иксов и игреков по иксам
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # class_number -- количество уникальных классов в y
        class_number = self.num_class

        # Что делает этот блок кода?
        # Вырезаем сердцевину из отсортировнных игреков
        cut_size = int(self.min_samples_split / 2) - 1
        splitted_sorted_y = sorted_y[cut_size:-
                                     cut_size] if cut_size != 0 else sorted_y
        # Правые индексы всех пар из различных соседних элементов в sorted_y
        r_border_ids = np.where(
            splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] \
            + (cut_size + 1)

        if len(r_border_ids) == 0:
            return float('+inf'), None

        # Что делает этот блок кода?
        # eq_el_count -- длины постоянных участков в splitted_sorted_y
        eq_el_count = r_border_ids - np.append([cut_size], r_border_ids[:-1])
        # one_hot_code -- таблица. one_hot_code[i, j]
        #     = Индикатор(r_border_ids[i] == j-тому классу)
        # В каждой строчке будет только одна единица
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1

        # Помножаем единицы из one_hot_code
        # на длины им соответствующих участков
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # Подправляем первую строчку в таблице,
        # прибавив к ней вектор, состоящий из
        # количеств элементов, встретившихся в
        # каждом классе до вырезания сердцевины
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Что делает этот блок кода?
        # Берем кумулятивные суммы по каждому столбцу
        # В i строке l_class_count будут лежать количества
        # элементов каждого класса встретившихся до r_border_ids[i]-го момента
        # Каждая строчка соответсвует разбиению:
        # в левом листе элементы до r_border_ids[i]-го, в правом -- остальные
        l_class_count = np.cumsum(class_increments, axis=0)
        # ------"------- после r_border_ids[i]-го момента
        # Каждая строчка соответсвует разбиению:
        # в левом листе элементы до r_border_ids[i]-го, в правом -- остальные
        r_class_count = np.bincount(
            sorted_y, minlength=class_number) - l_class_count
        # Размеры разбиений
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Считаем Gain для всех разбиений, выбираем наилучший
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmax(gs)

        # Что делает этот блок кода?
        # left_el_id -- индекс наилучшего разбиения
        left_el_id = l_sizes[idx][0]
        # возвращаем gain и искомый порог, находящийся
        # между sorted_x[left_el_id-1] и sorted_x[left_el_id]
        return gs[idx], (sorted_x[left_el_id - 1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        cond_len = len(y) <= self.min_samples_split
        cond_depth = depth == self.max_depth
        cond_class = len(np.unique(y)) == 1
        if cond_len or cond_depth or cond_class:
            # creating leaf
            classes_count = np.bincount(
                y, minlength=self.num_class).astype('float')
            probs = classes_count / np.sum(classes_count)
            self.tree[node_id] = (
                self.LEAF_TYPE, classes_count.argmax(), probs)
            return

        feature_ids = self.get_feature_ids(x.shape[1])
        threshold_list = list()
        gs_list = list()
        for curr_feature_id in feature_ids:
            curr_gss, curr_threshold = self.__find_threshold(
                x[:, curr_feature_id], y)
            gs_list.append(curr_gss)
            threshold_list.append(curr_threshold)
        best_gs = max(gs_list)
        best_feature_id = np.array(gs_list).argmax()
        best_threshold = threshold_list[best_feature_id]
        self.feature_importances_[best_feature_id] = best_gs
        left_x, right_x, left_y, right_y = self.__div_samples(
            x, y, best_feature_id, best_threshold)
        if len(left_y) == 0 or len(right_y) == 0:
            # creating leaf
            classes_count = np.bincount(
                y, minlength=self.num_class).astype('float')
            probs = classes_count / np.sum(classes_count)
            self.tree[node_id] = (
                self.LEAF_TYPE, classes_count.argmax(), probs)
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE,
                              best_feature_id, best_threshold)
        # left
        self.__fit_node(left_x, left_y, 2 * node_id + 1, depth + 1)
        # right
        self.__fit_node(right_x, right_y, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.empty(x.shape[1])
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [63]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [64]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [65]:
%time clf.fit(X_train, y_train)

CPU times: user 3.02 ms, sys: 4.63 ms, total: 7.65 ms
Wall time: 26.5 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [66]:
%time my_clf.fit(X_train, y_train)

CPU times: user 16.5 ms, sys: 5.02 ms, total: 21.5 ms
Wall time: 20.7 ms


## Проверка качества работы на wine

In [67]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8372183372183373

In [68]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8968253968253969

## Подготовка данных Speed Dating Data 

In [69]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb
'''КОПИПАСТ ИЗ КЛАССА'''
import pandas as pd
import numpy as np
!ls
df = pd.read_csv('./Speed Dating Data.csv', encoding='cp1251')
df = df.iloc[:, :97]
df.iid.nunique()
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df.drop_duplicates(subset=['iid']).gender.value_counts()
df.drop_duplicates(subset=['iid']).condtn.value_counts()
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'], axis=1)
df.drop_duplicates(subset=['iid']).age
df.drop_duplicates('iid').age.isnull().sum()
df = df.dropna(subset=['age'])
df.field_cd.isnull().sum()
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df = df.drop(['field'], axis=1)
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)

df.drop_duplicates('iid').mn_sat
df.drop_duplicates('iid').mn_sat.isnull().sum()
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)

df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.drop_duplicates('iid').tuition
df.drop_duplicates('iid').tuition.isnull().sum()
df.loc[:, 'tuition'] = df.tuition.fillna(-999)

df.drop_duplicates('iid').race.value_counts()
df.drop_duplicates('iid').age.isnull().sum()
df.drop_duplicates('iid').race
df.drop_duplicates('iid').imprace.isnull().sum()
df.drop_duplicates('iid').imprelig.isnull().sum()

df = df.dropna(subset=['imprelig', 'imprace'])

df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)

df.drop_duplicates('iid').loc[:, 'income']

df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)

df = df.dropna(subset=['date'])

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)

df = df.drop(['career'], axis=1)
df.loc[:, ['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
           'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga']
].isnull().sum()

df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)
df.drop_duplicates('iid').exphappy.isnull().sum()
df.drop_duplicates('iid').expnum.isnull().sum()
df = df.drop(['expnum'], axis=1)

feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 99)
temp.loc[idx,]
idx = ((temp.wave >= 6) & (temp.wave <= 9))

temp.loc[idx,]
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100
feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']

temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)

idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 90) & (temp.totalsum != 0)
temp.loc[idx,]

idx = ((temp.wave >= 6) & (temp.wave <= 9))
temp.loc[idx,]

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)

df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
    .drop(['gender'], axis=1) \
    .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
    .dropna()
df_female.columns = df_female.columns + '_f'

df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
df_pair.head()

X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

Speed Dating Data.csv hw2.ipynb


In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 9)

my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [71]:
X_train.shape

(3199, 63)

## Проверка скорости работы на Speed Dating Data 

In [83]:
%time  clf.fit(X_train, y_train)

CPU times: user 54.4 ms, sys: 1.26 ms, total: 55.6 ms
Wall time: 54.9 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [84]:
%time my_clf.fit(X_train, y_train)

CPU times: user 1.13 s, sys: 14.2 ms, total: 1.15 s
Wall time: 1.17 s


## Проверка качества работы на Speed Dating Data

In [85]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5385164376688641

In [86]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5421256117645684

## Задание 3

In [87]:
pipeline = Pipeline([('clf', MyDecisionTreeClassifier())])
X = pipeline.fit(X_train, y_train)
f1_score(y_pred=pipeline.predict(X_test), y_true=y_test, average='macro')

0.5421256117645684

## Задание 4

In [89]:
important_feature_ids_sklearn = clf.feature_importances_.argsort()[:10]
print("Sklearn importances")
print(df_pair.columns[1:][important_feature_ids_sklearn])

important_feature_ids_my_clf = my_clf.feature_importances_.argsort()[:10]
print()
print("My clf importances")
print(df_pair.columns[1:][important_feature_ids_my_clf])

Sklearn importances
Index(['date_f', 'go_out', 'samerace', 'career_c', 'race', 'fun2_1', 'imprace',
       'amb2_1', 'intel3_1', 'imprelig_f'],
      dtype='object')

My clf importances
Index(['sinc1_1', 'intel2_1', 'fun2_1', 'amb2_1', 'intel3_1', 'sinc3_1_f',
       'amb3_1_f', 'goal', 'attr3_1_f', 'imprace'],
      dtype='object')


## Задание 5

In [90]:
tree_params = {'max_depth': range(1,20), 'max_features': range(5,64)}

tree_grid = GridSearchCV(clf, tree_params,
                         cv=5, n_jobs=-1,
                            verbose=True)

tree_grid.fit(X_train, y_train)

tree_grid.best_params_, tree_grid.best_score_

Fitting 5 folds for each of 1121 candidates, totalling 5605 fits


[Parallel(n_jobs=-1)]: Done 852 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 4212 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done 5605 out of 5605 | elapsed:   47.3s finished


({'max_depth': 3, 'max_features': 36}, 0.8390121913097843)